# 1. Код Python

***В архиве нет доп. файлов, тк мне было удобнее распаковать архив во временную память***

In [567]:
import numpy as np
import pandas as pd
from zipfile import ZipFile
import urllib.request
from io import BytesIO

In [568]:
url = urllib.request.urlopen("http://archive.ics.uci.edu/ml/machine-learning-databases/00368/Facebook_metrics.zip")

In [569]:
with ZipFile(BytesIO(url.read())) as archive:
    dataframe = pd.read_csv(archive.open('dataset_Facebook.csv'), sep = ';')

Посчитаем необходимые значения для всех 3-х типов записей по каждому характеризующему столбцу. Для указанных характеристик 
хватит штатных методов pandas
Для 'Type', 'Paid' и 'Category' не имеет смысла считать все, кроме 'mode', т.к. переменная принимает известные значения

Т.к. у встроенного метода mode() есть ряд недостатков, компромиссный вариант - функция, возвращающая наиболее часто
встречающееся значение (значения, если есть несколько с одинаковой частотой) и, для оценки наглядности - его частоту. Допущение: мода засчитывается отсутствующей, если повторяющихся значений не было или частота была меньше sqrt(кол-во наблюдений) - чтобы исключить явные случайные совпадения

In [570]:
dataframe[1:5]

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
1,139441,Status,2,12,3,10,0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,3,12,3,3,0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,2,12,2,10,1,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,2,12,2,3,0,7244,13594,671,410,580,6228,3200,396,19,325,49,393


In [571]:
def get_mode(series, by_type):
    v = series.value_counts()
    if by_type == False:
        max_freq = max(v.values)
        m = list(v.loc[v.values == max_freq].index)
        return (max_freq, m) if max_freq>=20 else 'NaN'
    else:
        modes = []
        for typ in v.index.levels[0]:
            max_freq = max(v[typ].values)
            m = list(v[typ].loc[v[typ].values == max_freq].index)
            if max_freq>=5:
                modes.append({max_freq: m})
            else:
                modes.append('NaN')
        return modes

In [572]:
def get_stats(stats, series, func, by_type):
    shortcuts = {'mean': series.mean(), 'max': series.max(), 'min': series.min(), 'median': series.median(),
                 'mode': func(series, by_type)}
    for stat in shortcuts:
        stats[stat] = shortcuts[stat]
    return stats

In [573]:
outliers1 = ['Type', 'Paid', 'Category']
metrics = list(dataframe.columns.values)
index0 = ['mean', 'max', 'min', 'median', 'mode']
stats = pd.DataFrame(index = index0, columns = metrics)
for metric in metrics:
    if metric in outliers:
        stats.loc['mode'][metric] = get_mode(dataframe[metric], by_type = False)
    else:
        stats.loc[ :, metric] = get_stats( stats.loc[ :, metric], dataframe[metric], get_mode, False)

In [574]:
stats

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
mean,123194,NaN,NaN,7.038,4.15,7.84,NaN,13903.4,29585.9,920.344,798.772,1415.13,16766.4,6585.49,609.986,7.482,177.946,27.2661,212.12
max,139441,NaN,NaN,12,7,23,NaN,180480,1110282,11452,11328,19779,1107833,51456,4376,372,5172,790,6334
min,81370,NaN,NaN,1,1,1,NaN,238,570,9,9,9,567,236,9,0,0,0,0
median,129600,NaN,NaN,7,4,9,NaN,5281,9051,625.5,551.5,851,6255.5,3417,412,3,101,19,123.5
mode,NaN,"(426, [Photo])","(215, [1])","(60, [10])","(82, [7])","(105, [3])","(360, [0.0])",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"(106, [0])",NaN,"(20, [14.0, 13.0])",NaN


Те же статистики отдельно для каждого типа переменной "Type". В данном случае моду для 'Category' также можно не считать:

In [575]:
seq = [(stat, type) for stat in index1 for type in list(dataframe['Type'].unique())]
multiindex = pd.MultiIndex.from_tuples(seq, names = ['Stat', 'Type'])
specs_stats = pd.DataFrame(index = multiindex, columns = metrics)
specs_dataframe = dataframe.groupby('Type')

In [576]:
for metric in metrics:
    if metric == 'Type':
        None
    elif metric == 'Category' or 'Paid':
        specs_stats.loc[:, metric]['mode'] = get_mode(specs_dataframe[metric], by_type = True)
    else:
        specs_stats.loc[ :, metric] = get_stats( specs_stats.loc[ :, metric], specs_dataframe[metric], get_mode, True)

In [577]:
specs_stats

Page total likes Type    Category  Post Month Post Weekday  \
Stat   Type                                                                
mean   Photo               NaN  NaN         NaN         NaN          NaN   
       Status              NaN  NaN         NaN         NaN          NaN   
       Link                NaN  NaN         NaN         NaN          NaN   
       Video               NaN  NaN         NaN         NaN          NaN   
max    Photo               NaN  NaN         NaN         NaN          NaN   
       Status              NaN  NaN         NaN         NaN          NaN   
       Link                NaN  NaN         NaN         NaN          NaN   
       Video               NaN  NaN         NaN         NaN          NaN   
min    Photo               NaN  NaN         NaN         NaN          NaN   
       Status              NaN  NaN         NaN         NaN          NaN   
       Link                NaN  NaN         NaN         NaN          NaN   
       Video               NaN  NaN         NaN         NaN          NaN   
median Photo               NaN  NaN         NaN         NaN          NaN   
       Status              NaN  NaN         NaN         NaN          NaN   
       Link                NaN  NaN         NaN         NaN          NaN   
       Video               NaN  NaN         NaN         NaN          NaN   
mode   Photo               NaN  NaN   {20: [1]}   {6: [12]}     {5: [6]}   
       Status   {17: [124940]}  NaN  {185: [1]}  {49: [10]}    {71: [7]}   
       Link                NaN  NaN   {38: [2]}  {12: [12]}  {9: [5, 7]}   
       Video               NaN  NaN    {7: [1]}         NaN          NaN   

                Post Hour          Paid Lifetime Post Total Reach  \
Stat   Type                                                         
mean   Photo          NaN           NaN                       NaN   
       Status         NaN           NaN                       NaN   
       Link           NaN           NaN                       NaN   
       Video          NaN           NaN                       NaN   
max    Photo          NaN           NaN                       NaN   
       Status         NaN           NaN                       NaN   
       Link           NaN           NaN                       NaN   
       Video          NaN           NaN                       NaN   
min    Photo          NaN           NaN                       NaN   
       Status         NaN           NaN                       NaN   
       Link           NaN           NaN                       NaN   
       Video          NaN           NaN                       NaN   
median Photo          NaN           NaN                       NaN   
       Status         NaN           NaN                       NaN   
       Link           NaN           NaN                       NaN   
       Video          NaN           NaN                       NaN   
mode   Photo     {6: [3]}   {16: [0.0]}                       NaN   
       Status   {89: [3]}  {306: [0.0]}                       NaN   
       Link    {10: [10]}   {35: [0.0]}                       NaN   
       Video          NaN           NaN                       NaN   

              Lifetime Post Total Impressions Lifetime Engaged Users  \
Stat   Type                                                            
mean   Photo                              NaN                    NaN   
       Status                             NaN                    NaN   
       Link                               NaN                    NaN   
       Video                              NaN                    NaN   
max    Photo                              NaN                    NaN   
       Status                             NaN                    NaN   
       Link                               NaN                    NaN   
       Video                              NaN                    NaN   
min    Photo                              NaN                    NaN   
       Status                 

In [578]:
dataframe

,Page total likes,Type,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,Lifetime Post Consumptions,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Total Interactions
0,139441,Photo,2,12,4,3,0,2752,5091,178,109,159,3078,1640,119,4,79,17,100
1,139441,Status,2,12,3,10,0,10460,19057,1457,1361,1674,11710,6112,1108,5,130,29,164
2,139441,Photo,3,12,3,3,0,2413,4373,177,113,154,2812,1503,132,0,66,14,80
3,139441,Photo,2,12,2,10,1,50128,87991,2211,790,1119,61027,32048,1386,58,1572,147,1777
4,139441,Photo,2,12,2,3,0,7244,13594,671,410,580,6228,3200,396,19,325,49,393
5,139441,Status,2,12,1,9,0,10472,20849,1191,1073,1389,16034,7852,1016,1,152,33,186
6,139441,Photo,3,12,1,3,1,11692,19479,481,265,364,15432,9328,379,3,249,27,279
7,139441,Photo,3,12,7,9,1,13720,24137,537,232,305,19728,11056,422,0,325,14,339
8,139441,Status,2,12,7,3,0,11844,22538,1530,1407,1692,15220,7912,1250,0,161,31,192
9,139441,Photo,3,12,6,10,0,4694,8668,280,183,250,4309,2324,199,3,113,26,142


Из данного массива можно получить много информации. Самый популярный объект в выборке является публикацией на Фейсбуке. Тк Фейсбук использует сложный алгоритм сортировки постов, время публикации поста можно не учитывать при оценке его популярности. 
Нас интересует популярность поста у всех пользователей, а не только у пользователей, подписанных на страницу. Пост появляется в новостной ленте у новых пользователей если он: 1) набирает много лайков, репостов или комментариев 2) лайкнул/прокомментировал кто-то из друзей 3) это оплаченная реклама (видимо, метрика 'Paid'). Т.е., если, при прочих допускаемо равных, у одного поста в метрике 'Paid' стоит '0', а у второго '1', первый был популярнее. 

Таким образом, отправной характеристикой популярности поста можно считать сумму метрик 'like'+'share'+'comment'. Затем, можно
сверить метрику 'Paid'. Получим:

In [579]:
dataframe['Popularity'] = dataframe['like'] + dataframe['share'] + dataframe['comment']

In [580]:
dataframe[['Lifetime Post Total Reach', 'Paid', 'Popularity']].sort_values('Popularity', ascending = False)[0:5]

,Lifetime Post Total Reach,Paid,Popularity
244,180480,1,6334
379,105632,1,2177
168,41984,0,1974
460,55520,0,1873
349,81856,0,1806


Как видим, наиболее популярным постом был №244, он же имел самое высокое кол-во просмотров пользователями (по предположению,
уникальными). Первый следующий за ним неоплаченный пост №168 имел в 3,2 раза меньше просмотров и в 4,3 раза меньше показов. Можно найти линейный фактор, очень грубо характеризующий увеличение вероятности поста быть показанным, если он был оплачен, либо для этой же цели прогнать простую регрессию (я посчитала это лишним), чтобы посмотреть не был ли пост №244 явным выбросом и, следовательно, на самом деле более популярным, чем пост №244:

In [581]:
factor = dataframe[dataframe['Paid']==1][['Popularity',
                    'Lifetime Post Total Reach']].mean() / dataframe[dataframe['Paid']==0][['Popularity',
                    'Lifetime Post Total Reach']].mean()
factor

Popularity                   1.473081
Lifetime Post Total Reach    1.627526
dtype: float64

Как видим, средняя популярность оплаченных постов выше в полтора раза, чем у неоплаченных. Следовательно, вероятностью того, что
пост №168 мог быть самым популярным, можно предварительно пренебречь. Спасибо, если вы вдруг дочитали.

# 2. Метрики для Fantasy

*Сразу прошу прощения, если где-то была использована неправильная терминология.

В описании задания не были специально указаны цели, под которые должны подбираться метрики, поэтому структура анализа такова:
1. определить задачи, которые преследует оптимизация
2. подобрать метрики, по которым можно определить, насколько успешно текущая конфигурация решает эти задачи

**Задачи.**
Я предполагаю 4 стандартные задачи (без цифр):
1. Увеличение просмотров страниц раздела по переходам из основных источников: поисковиков и социальных сетей
2. Повышение конверсии пользователей в зарегистрировавшихся после посещения данного раздела портала
2. Повышении конверсии пользователей раздела в активных
3. Поддержание активности пользователя

Насколько я поняла, система призов и капиталов еще не введена, поэтому пока в качестве активного будет рассматриваться пользователь, участвующий
в создании и изменении фэнтези-команд и просматривающий от 15 страниц раздела, в среднем, за 2-3 дня.

**Метрики**
1. Метрики для оценки просмотров
    1. по переходам из поисковой выдачи:
 - поисковые запросы
 - какие страницы ассоциируются с наибольшей/наименьшей глубиной просмотров / процентом отказов (тк влияет на ранжирование)
    2. соц.сети: 
 - из каких соц. сетей происходит переход
 - в каких соц. сетях расшаривается больше всего записей
2. Аккаунты каких соц. сетей используются как вспомогательные при регистрации из раздела / после просмотра страниц раздела
    1. Тематика страниц раздела, просмотренных зарегистрировавшимися сразу после этого пользователями (ВЗП)
    2. Какие страницы ВЗП доскролливают до конца, а с каких переходят 
    3. Среднее количество страниц, просмотренных ВЗП
    4. Ресурсы, с которых приходят ВЗП (почтовые рассылки, рассылки-боты, другие разделы основного портала, соц. сети, мобильные приложения)
    5. Сегментация зарегистрировавшися после просмотра раздела пользователей по географии
    6. Соотношение прямых переходов на страницу раздела (например, пользователь сделал закладку в браузере или получил прямую ссылку от друга) / переходов по поисковому запросу или из постов, размещенных в соц. сетях для ВЗП
3. Те же что и в пункте 2, кроме F, но для пользователей, помеченных впоследствии, как "активные" (ВАП) (тк логично предположить, что в раздел Fantasy заходят пользователи, ужзе зарегистрированные на портале sports.ru)
    1. Страницы выхода для авторизированных пользователей, просматривающих страницы время от времени, но не ставших ВАП
4. Те же, что в пункте 3

# 3. Основные различия между аналитикой веб-сайта и мобильных приложений

Под **аналитикой веб-сайта** подразумевается аналитика данных о действиях пользователя непосредственно на сайте портала, будь то версия сайта, адаптированная под мобильную платформу или полная версия.
**Аналитика мобильных приложений** - аналитика данных, полученных, собственно, из мобильных приложений (прошу прощения), из которых возможен доступ к контенту портала.

1. Мобильные приложения используются, по большей части, существующими пользователями (исключение для соц. сетей), в то время как доля привлеченных новых пользователей будет выше для веб-сайта. Это влияет на выбор метрик для оптимизации: например, для сайта будет иметь значение кол-во просмотренных страниц, а для моб. приложения - длина сессии и кол-во совершенных действий.
2. При аналитике мобильного приложения большее внимание следует обращать на использование элементов интерфейса пользователями и удобство.
3. Из ряда моб. приложений можно получить дополнительные сведения о пользователе: использование соц. сетей, география.
